In [1]:
# import necessary packages
import os
import pandas as pd

# Influencer Marketing Report: CPA and Conversion Rates

## Load data

Our data comes in the form of Excel files summarizing performance of influencer marketing campaigns. Each file has the following sheets:
* `Conversions`: A pivot table showing the number of conversions for each influencer on different days
* `Posts`: A table recording each post as well as the likes, comments, shares, etc. It also contains information about the influencer (number of followers, cost per post, etc).

We'll be using [Pandas]() to work with this tabular data. Because the Excel files have the extension `.xlsx`, we also need to have the `OpenPyXL` library installed. Pandas uses this as an engine to read the data.

In [2]:
DATA_DIR = os.path.abspath('./../data')
if not os.path.exists('reports'):
    os.mkdir('reports')

### Posts

In [3]:
report_path = os.path.join(DATA_DIR, 'influencer-data-2019Q1.xlsx')
posts = pd.read_excel(report_path, engine='openpyxl', sheet_name=1, index_col=0)

In [4]:
posts.head(2)

,Date,Influencer,Followers,Follower group,Cost,Likes,Comments,Shares,Clicks
0,2019-01-05,Rachel Kelly,1029,Group 1 (1000-2000),80,336,14,17,52
1,2019-01-10,Rachel Kelly,1029,Group 1 (1000-2000),80,279,11,14,42


We want to convert the "Date" column to a Pandas `datetime` object. We'll address this more later!

In [5]:
posts['Date'] = pd.to_datetime(posts['Date'])

### Conversions

In [6]:
conversions = pd.read_excel(report_path, engine='openpyxl', sheet_name=0)

In [7]:
conversions.head(2)

,influencer,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31
0,Albert Taylor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,5.0,NaN
1,Amanda Campbell,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


We see that each column is a specific day and that each row is a specific influencer. The values in the cells denote the number of conversions attributed to the influencer on that day.

We see the value `NaN` in a lot of these cells, which is a Pandas data type that means "Not a number". In other words, these cells are empty. For us, that means that there were zero conversions that day, so we can fill them with zeros. 

In [8]:
conversions.fillna(0).head(2)

,influencer,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31
0,Albert Taylor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,5.0,0.0
1,Amanda Campbell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Merge tables


To compute our metrics, we need to merge the two tables together. This is because we need to look at `conversions / cost` (CPA) as well as `conversions / clicks` (conversion rate), and this information is spread across two tables.

Thus, we want a new table that looks like `posts` but with a `conversions` column.

### "Melt" the conversions table

To match up the tables and merge them, we need to change the structure of the `conversions` table. It's currently a pivot table - we need to "melt" it back to a long-form table.

Instead of a column for each day and a row for each influencer, we want three columns:
* influencer
* date
* conversions

This structure matches the structure of the `posts` table.

In [9]:
conversions.head(2)

,influencer,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-03-22,2019-03-23,2019-03-24,2019-03-25,2019-03-26,2019-03-27,2019-03-28,2019-03-29,2019-03-30,2019-03-31
0,Albert Taylor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,5.0,NaN
1,Amanda Campbell,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
melted = conversions.melt(id_vars='influencer', var_name='Date', value_name='Conversions').fillna(0)

In [11]:
melted['Date'] = pd.to_datetime(melted['Date'])

In [12]:
df = posts.merge(melted, left_on=['Date', 'Influencer'], right_on=['Date', 'influencer'])

In [13]:
df.head()

,Date,Influencer,Followers,Follower group,Cost,Likes,Comments,Shares,Clicks,influencer,Conversions
0,2019-01-05,Rachel Kelly,1029,Group 1 (1000-2000),80,336,14,17,52,Rachel Kelly,1.0
1,2019-01-10,Rachel Kelly,1029,Group 1 (1000-2000),80,279,11,14,42,Rachel Kelly,1.0
2,2019-01-16,Rachel Kelly,1029,Group 1 (1000-2000),80,390,3,19,47,Rachel Kelly,1.0
3,2019-01-26,Rachel Kelly,1029,Group 1 (1000-2000),80,373,14,19,64,Rachel Kelly,1.0
4,2019-01-28,Rachel Kelly,1029,Group 1 (1000-2000),80,571,9,27,34,Rachel Kelly,0.0


### Wrap this up into a function

So far we have 
1. Loaded the data
2. Melted the conversions table
3. Merged the two tables


We want to be able to do this with more files in the future, so let's put all that code in a function.

In [14]:
def load_and_preprocess(filepath):
    posts = pd.read_excel(filepath, engine='openpyxl', sheet_name=1, index_col=0)
    posts['Date'] = pd.to_datetime(posts['Date'])
    
    conversions = pd.read_excel(filepath, engine='openpyxl', sheet_name=0)
    melted = conversions.melt(id_vars='influencer', var_name='Date', value_name='Conversions').fillna(0)
    melted['Date'] = pd.to_datetime(melted['Date'])
    df = posts.merge(melted, left_on=['Date', 'Influencer'], right_on=['Date', 'influencer'])
    df['month'] = df['Date'].dt.strftime('%B %Y')
    return df

In [15]:
df = load_and_preprocess(report_path)

## Metrics

### Cost per Acquisition

In [16]:
def cost_per_acquisition(df):
    if df['Conversions'].sum() > 0:
        return df['Cost'].sum() / df['Conversions'].sum()

#### CPA for each influencer

We can group our dataframe by various columns and apply that `cost_per_acquisition` function to each group. First, let's try doing this by influencer. 

In [17]:
cpa_influencer = df.groupby('influencer').apply(cost_per_acquisition)
cpa_influencer

influencer
Albert Taylor               29.090909
Amanda Campbell            800.000000
Angela Martinez            355.555556
Brandon Gonzalez           106.666667
Christopher Washington     116.666667
Eugene White                40.000000
George Smith                80.000000
Gregory Cook               171.428571
Harold Henderson           200.000000
Julia Reed                  32.000000
Linda James                 88.888889
Lois Carter                 35.200000
Lori Young                1166.666667
Louise Garcia               88.888889
Marie Lee                   40.000000
Mildred Peterson            53.333333
Nancy Watson                78.750000
Nicole Stewart              33.333333
Paul Green                 100.000000
Rachel Kelly                70.588235
Ruby Perry                 119.047619
Samuel Parker              142.857143
Sharon Davis                44.000000
Tina Wright                 52.500000
Victor Evans               114.285714
dtype: float64

#### CPA for each month

In [18]:
df.groupby('month').apply(cost_per_acquisition)

month
February 2019    87.080745
January 2019     98.650307
March 2019       95.244755
dtype: float64

### Conversion rate

In [19]:
def conversion_rate(df):
    if df['Clicks'].sum() > 0:
        return df['Conversions'].sum() / df['Clicks'].sum()
    else:
        return None

## Report function

Let's write a function that generates a report after grouping on some columns we define. This will output something with:
* Posts
* Cost
* Clicks
* Conversions
* Conversion Rate
* CPA

For each group.

In [20]:
def generate_report(df, grouping_column):
    cpa = df.groupby(grouping_column).apply(cost_per_acquisition).rename('CPA').to_frame()
    conv = df.groupby(grouping_column).apply(conversion_rate).rename('Conversion Rate').to_frame()
    sums = df.groupby(grouping_column)[['Cost', 'Clicks', 'Conversions']].sum()
    num_posts = df.groupby(grouping_column).apply(len).rename('Number of Posts').to_frame()
    
    
    out = num_posts.join(sums).join(conv).join(cpa)
    return out

In [21]:
df = load_and_preprocess(report_path)
report_df = generate_report(df, ['Influencer'])

report_df

,Number of Posts,Cost,Clicks,Conversions,Conversion Rate,CPA
Influencer,,,,,,
Albert Taylor,8,960,2257,33.0,0.014621,29.090909
Amanda Campbell,25,4000,563,5.0,0.008881,800.000000
Angela Martinez,20,3200,609,9.0,0.014778,355.555556
Brandon Gonzalez,6,960,570,9.0,0.015789,106.666667
Christopher Washington,15,2100,860,18.0,0.020930,116.666667
Eugene White,20,800,1176,20.0,0.017007,40.000000
George Smith,15,1200,957,15.0,0.015674,80.000000
Gregory Cook,15,2400,674,14.0,0.020772,171.428571
Harold Henderson,15,1800,415,9.0,0.021687,200.000000


We can use this to e.g. the five influencers with the highest CPA:

In [22]:
report_df.sort_values('CPA', ascending=False).head()

,Number of Posts,Cost,Clicks,Conversions,Conversion Rate,CPA
Influencer,,,,,,
Lori Young,25,3500,357,3.0,0.008403,1166.666667
Amanda Campbell,25,4000,563,5.0,0.008881,800.000000
Angela Martinez,20,3200,609,9.0,0.014778,355.555556
Harold Henderson,15,1800,415,9.0,0.021687,200.000000
Gregory Cook,15,2400,674,14.0,0.020772,171.428571


## Write to Excel

Now, we can write this back to excel!

In [23]:
def write_report_to_excel(input_path, output_path='report.xlsx'):
    
    posts = pd.read_excel(input_path, 'Posts', engine='openpyxl')
    posts = pd.read_excel(input_path, 'Conversions', engine='openpyxl')

    df = load_and_preprocess(input_path)
    
    by_influencer = generate_report(df, ['Influencer'])
    by_month = generate_report(df, ['month'])
    by_influencer_and_month = generate_report(df, ['Influencer', 'month'])
    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(output_path, engine='openpyxl')
    
    # Write each dataframe to a different worksheet.
    
    by_influencer.to_excel(writer, sheet_name='Report - Influencer')
    by_month.to_excel(writer, sheet_name='Report - Month')
    by_influencer_and_month.to_excel(writer, sheet_name='Report - Influencer and Month')
    
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [24]:
write_report_to_excel(report_path, 'reports/influencer-report-2019Q1.xlsx')